## Download html files

request,bsでは動的コンテンツをとってこれないのでseleniumを使う

In [ ]:
from selenium import webdriver
from time import sleep
import requests

In [3]:
url="https://toho-vote.info/result"

In [ ]:
driver = webdriver.Chrome('ChromeDriverのディレクトリ + chromedriver')

In [5]:
driver.get(url)
for button in []:
    driver.find_element_by_xpath(button).click()
    time.sleep(1)

<Response [200]>

In [ ]:
driver.quit()

第16回以前は静的ファイル

In [142]:
url_tr="https://toho-vote.info/results"

## htmlからcsvファイル作成(scraping)

In [1]:
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート
ddir="data"

In [5]:
filename_tr="/18/第18回東方Project人気投票_人妖"
filename=ddir+filename_tr+".html"

In [5]:
with open(ddir+filename,"r") as fp:
    htmltext=fp.read()
soup = BeautifulSoup(htmltext, "html.parser")

ブラウザのデバッガ
document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
の結果から

In [4]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.div

<div class="_title_en_1h8e9_51">Characters</div>

In [5]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header

<header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header>

In [6]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]

In [7]:
table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div


In [8]:
table_header=table.find_all("th")
table_contents=table.find_all("tr")

In [9]:
table_header

[<th>順位</th>,
 <th class="_rank_prev_1orw9_133">前回</th>,
 <th class="_rank_prev2_1orw9_136">前々</th>,
 <th class="_name_1orw9_101">名前</th>,
 <th>ポイント</th>,
 <th>一推し</th>,
 <th>コメント</th>,
 <th>支援作品</th>]

In [12]:
outdata=get_tabledata(table_contents)

In [14]:
outdata[0]=outheader
outdata[:2]

[[], ['1', '5', '5', 'フランドール・スカーレット', '16,723', '3,730', '3,633', '19']]

### output

## 第16回以前の投票結果

文字コード

In [85]:
!nkf -g */*/*.html

data/1/東方シリーズキャラ人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.html: CP932
data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.html: EUC-JP
data/11/アンケート投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/経緯報告書 - 第11回 東方Project 人気投票.html: UTF-8
data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/作品部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/13/アンケート集計結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/音楽部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/作品部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/人妖部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/14/アンケート部門集計結果 - 第14回 東方Project 人気投票.html: UTF-8
da

UTF-8, EUC＿JP,CP932が混在している

In [327]:
import csv

def get_tabledata(table):
    return [[t.string for t in tr.find_all("td") ]+[t.string for t in tr.find_all("th") ] for tr in table ]

#def get_tableheader(table):
#    return [[t.string for t in tr.find_all("th")] for tr in table]  

def openhtml(ddir,filename_tr):
    filename=ddir+filename_tr+".html"
    # dirty hack for character code
    try:
        with open(filename,"r") as fp:
            htmltext=fp.read()
    except:
        try:
            with open(filename,"r",encoding='euc_jp') as fp:
                htmltext=fp.read()
        except:
            try:
                with open(filename,"r",encoding='CP932') as fp:
                    htmltext=fp.read()
            except:
                print("cannot read"+filename)
    return htmltext
    
def souptable2csv(table,writedown=False,filename_tr=""):
#    table_header=table.find_all("th")
#    outheader=get_tableheader(table_header)
    table_contents=table.find_all("tr")
    outdata  =get_tabledata(table_contents)
    outdata =[x for x in outdata if len(x)>1 ] #bad 
    if(writedown):
        with open(ddir+filename_tr+".csv", 'w') as fpw:
            writer = csv.writer(fpw, lineterminator='\n')
            writer.writerows(outdata)

def converthtml2csv(filename_tr,writedown=False):
    htmltext=openhtml(ddir,filename_tr)
    soup = BeautifulSoup(htmltext, "html.parser")
    table=soup 
    souptable2csv(table,writedown,filename_tr)

In [325]:
filename_tr="/1/東方シリーズキャラ人気投票 [Vote Top]"
htmltext=openhtml(ddir,filename_tr)
soup = BeautifulSoup(htmltext, "html.parser")
table=soup 
table_contents=table.find_all("tr")
outdata =get_tabledata(table_contents)
outdata =[x for x in outdata if len(x)>1 ]

In [326]:
outdata


[['ランク', '投票', '項目', '投票数', '投票理由', 'グラフ'],
 ['十六夜咲夜', None, None, '1位', None, '95'],
 ['霧雨 魔理沙', None, None, '-', None, '95'],
 ['博麗 霊夢', None, None, '3位', None, '62'],
 ['チルノ', None, None, '4位', None, '51'],
 ['パチュリー・ノーレッジ', None, None, '5位', None, '46'],
 ['橙', None, None, '6位', None, '43'],
 ['フランドール・スカーレット', None, None, '7位', None, '32'],
 ['レミリア・スカーレット', None, None, '8位', None, '28'],
 ['アリス・マーガトロイド', None, None, '9位', None, '27'],
 ['魅魔', None, None, '10位', None, '14'],
 ['紅 美鈴', None, None, '11位', None, '9'],
 ['小兎姫', None, None, '12位', None, '7'],
 ['めい＆まい', None, None, '13位', None, '5'],
 ['ルーミア', None, None, '-', None, '5'],
 ['幽香', None, None, '-', None, '5'],
 ['エーリッヒ', None, None, '16位', None, '3'],
 ['エレン・ふわふわ頭・オーレウス', None, None, '-', None, '3'],
 ['朝倉 理香子', None, None, '-', None, '3'],
 ['ビビット(VIVIT)', None, None, '19位', None, '2'],
 ['レティ・ホワイトロック', None, None, '-', None, '2'],
 ['玄爺', None, None, '-', None, '2'],
 ['里香', None, None, '-', None, '2'],
 ['マイ', None, None

In [328]:
import glob
htmlfiles=glob.glob("data/*/*.html")

for f in htmlfiles:
    filename_tr=f[4:]
    filename_tr=filename_tr.split(".")[0]
    #print (filename_tr)
    converthtml2csv(filename_tr,True)

In [329]:
!wc data/*/*.csv

    26     31    736 data/1/東方シリーズキャラ人気投票 [Vote Top].csv
    75    163   2699 data/1/東方シリーズ音楽人気投票 [Vote Top].csv
    75    163   2699 data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv
   160    226   8136 data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.csv
   432    824  27980 data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.csv
   216    228   5792 data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.csv
   268    268  10167 data/11/アンケート投票結果 - 第11回 東方Project 人気投票.csv
   173    240   8784 data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.csv
   916   1309  26671 data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.csv
     0      0      0 data/11/経緯報告書 - 第11回 東方Project 人気投票.csv
   260    260  11161 data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.csv
   476    859  27858 data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.csv
    42     48   1941 data/12/作品部門投票結果 - 第12回 東方Project 人気投票.csv
   170    238   9351 data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.csv
   274    274  11756 data/13/アンケート集計結果 - 第13回 東方Project 人気投票.csv
   438   1003  24740 data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.csv
   4

In [293]:
!wc data/*/*.csv

    76     85   1242 data/1/東方シリーズキャラ人気投票 [Vote Top].csv
   225    400   4958 data/1/東方シリーズ音楽人気投票 [Vote Top].csv
   225    400   4958 data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv
   160    226   8136 data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.csv
   432    824  27980 data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.csv
   216    228   5792 data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.csv
   268    268  10167 data/11/アンケート投票結果 - 第11回 東方Project 人気投票.csv
   173    240   8784 data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.csv
   916   1309  26671 data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.csv
     0      0      0 data/11/経緯報告書 - 第11回 東方Project 人気投票.csv
   260    260  11161 data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.csv
   476    859  27858 data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.csv
    42     48   1941 data/12/作品部門投票結果 - 第12回 東方Project 人気投票.csv
   170    238   9351 data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.csv
   274    274  11756 data/13/アンケート集計結果 - 第13回 東方Project 人気投票.csv
   438   1003  24740 data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.csv
   4

In [318]:
!head -2 data/1/*音楽*.csv

==> data/1/東方シリーズ音楽人気投票 [Vote Top].csv <==
""
ランク,投票,項目,投票数,投票理由,グラフ

==> data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv <==
""
ランク,投票,項目,投票数,投票理由,グラフ


第一回は特別な処理が必要になる

In [261]:
import pandas as pd

In [330]:
vote1_char=pd.read_csv("data/1/東方シリーズキャラ人気投票 [Vote Top].csv")
vote1_music=pd.read_csv("data/1/東方シリーズ音楽人気投票 [Vote Top].csv")

In [331]:
vote1_char

,ランク,投票,項目,投票数,投票理由,グラフ
0,十六夜咲夜,NaN,NaN,1位,NaN,95
1,霧雨 魔理沙,NaN,NaN,-,NaN,95
2,博麗 霊夢,NaN,NaN,3位,NaN,62
3,チルノ,NaN,NaN,4位,NaN,51
4,パチュリー・ノーレッジ,NaN,NaN,5位,NaN,46
5,橙,NaN,NaN,6位,NaN,43
6,フランドール・スカーレット,NaN,NaN,7位,NaN,32
7,レミリア・スカーレット,NaN,NaN,8位,NaN,28
8,アリス・マーガトロイド,NaN,NaN,9位,NaN,27
9,魅魔,NaN,NaN,10位,NaN,14


In [343]:
vote1_char=vote1_char.dropna(axis=1,how="any")
vote1_char=vote1_char.replace('十六夜咲夜', '十六夜 咲夜',regex=True)

,ランク,投票数,グラフ
0,十六夜 咲夜,1位,95
1,霧雨 魔理沙,-,95
2,博麗 霊夢,3位,62
3,チルノ,4位,51
4,パチュリー・ノーレッジ,5位,46
5,橙,6位,43
6,フランドール・スカーレット,7位,32
7,レミリア・スカーレット,8位,28
8,アリス・マーガトロイド,9位,27
9,魅魔,10位,14


In [337]:
vote1_music.dropna(axis=1,how="any")

,ランク,投票数,グラフ
0,亡き王女の為のセプテット,1位,81
1,U.N.オーエンは彼女なのか？,2位,77
2,明治十七年の上海アリス,3位,49
3,上海紅茶館 ～ Chinese Tea,4位,47
4,月時計 ～ ルナ・ダイアル,5位,45
...,...,...,...
69,東方妖恋談,-,1
70,博麗 ～Eastern Wind,-,1
71,夢幻回廊,-,1
72,夢想時空,-,1


In [143]:
_char =pd.read_csv("data/2/第２回東方シリーズ人気投票 キャラ部門結果.csv")
_music=pd.read_csv("data/2/第２回東方シリーズ人気投票 音楽部門結果.csv")
_danmaku=pd.read_csv("data/2/第２回東方シリーズ人気投票 弾幕部門結果.csv")
vote2={"char":_char,"music":_music,"danmaku":_danmaku}

In [132]:
#vote_result[1]=vote2
vote_result.append(vote2)

In [144]:
_char

,順位,名前,Unnamed: 2,投票理由
0,1,十六夜 咲夜,564（372）,[コメント]
1,2,霧雨 魔理沙,532（398）,[コメント]
2,3,博麗 霊夢,293（203）,[コメント]
3,4,プリズムリバー三姉妹,260（207）,[コメント]
4,5,魂魄 妖夢,254（198）,[コメント]
5,6,八雲 藍,236（192）,[コメント]
6,7,パチュリー・ノーレッジ,195（153）,[コメント]
7,8,アリス・マーガトロイド,184（145）,[コメント]
8,9,紅 美鈴,164（141）,[コメント]
9,10,レミリア・スカーレット,162（138）,[コメント]


In [136]:
vote_result[0]["char"].columns=["名前","順位","投票数"]

In [137]:
vote_result[0]["char"]

,名前,順位,投票数
0,十六夜咲夜,1位,95.0
1,霧雨 魔理沙,-,95.0
2,博麗 霊夢,3位,62.0
3,チルノ,4位,51.0
4,パチュリー・ノーレッジ,5位,46.0
5,橙,6位,43.0
6,フランドール・スカーレット,7位,32.0
7,レミリア・スカーレット,8位,28.0
8,アリス・マーガトロイド,9位,27.0
9,魅魔,10位,14.0


In [141]:
pd.merge(vote_result[0]["char"],vote_result[1]["char"],on="名前",how="outer")

,名前,順位_x,投票数,順位_y,Unnamed: 2,投票理由
0,十六夜咲夜,1位,95.0,NaN,NaN,NaN
1,霧雨 魔理沙,-,95.0,2.0,532（398）,[コメント]
2,博麗 霊夢,3位,62.0,3.0,293（203）,[コメント]
3,チルノ,4位,51.0,16.0,65（50）,[コメント]
4,パチュリー・ノーレッジ,5位,46.0,7.0,195（153）,[コメント]
5,橙,6位,43.0,12.0,142（120）,[コメント]
6,フランドール・スカーレット,7位,32.0,15.0,93（85）,[コメント]
7,レミリア・スカーレット,8位,28.0,10.0,162（138）,[コメント]
8,アリス・マーガトロイド,9位,27.0,8.0,184（145）,[コメント]
9,魅魔,10位,14.0,21.0,19（15）,[コメント]


In [149]:
char16=pd.read_csv("data/16/人妖部門投票結果 - 第16回 東方Project 人気投票.csv")
char16

,順位,前回,前々,名前,ポイント,一押し,コメント,支援作品,Unnamed: 8
0,1,3,4,魂魄 妖夢,9841,2290,1360,35,詳細情報
1,2,1,1,博麗 霊夢,8913,1777,1224,21,詳細情報
2,3,4,3,古明地 こいし,8865,1591,1176,17,詳細情報
3,4,2,2,霧雨 魔理沙,8626,1587,1172,17,詳細情報
4,5,6,6,フランドール・スカーレット,8497,1607,1207,20,詳細情報
...,...,...,...,...,...,...,...,...,...
193,191,183,180,毘沙門天,7,0,0,0,詳細情報
194,195,185,185,煙々羅,5,0,2,0,詳細情報
195,196,176,166,ヤマネ,4,0,1,0,詳細情報
196,196,179,180,座敷童,4,0,0,0,詳細情報


In [253]:
!head -1 data/*/*.csv

==> data/1/東方シリーズキャラ人気投票 [Vote Top].csv <==
""

==> data/1/東方シリーズ音楽人気投票 [Vote Top].csv <==
""

==> data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv <==
""

==> data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.csv <==
,,,名前,,,

==> data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.csv <==
,,,名前,,登場作品,

==> data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.csv <==
男性,12966人,88.98%,

==> data/11/アンケート投票結果 - 第11回 東方Project 人気投票.csv <==
11,0.11%,～9歳

==> data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.csv <==
順位,前回,前々,名前,ポイント,一押し,コメント,支援リンク,　

==> data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.csv <==
順位,前回,前々,名前,ポイント,一押し,コメント,　

==> data/11/経緯報告書 - 第11回 東方Project 人気投票.csv <==

==> data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.csv <==
19,0.11%,～9歳

==> data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.csv <==
順位,前回,前々,名前,ポイント,一押し,コメント,　

==> data/12/作品部門投票結果 - 第12回 東方Project 人気投票.csv <==
順位,名前,ポイント,一押し,コメント,　

==> data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.csv <==
順位,前回,前々,名前,ポイント,一押し,コメント,支援作品,　

==> data/13/アンケート集計結果 - 第13回 東方Project 人気投票.csv <==
16,0.11%,～9歳

==> data/13/ベストパートナー部